Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB10

Use reinforcement learning to devise a tic-tac-toe player.

### Deadlines:

* Submission: [Dies Natalis Solis Invicti](https://en.wikipedia.org/wiki/Sol_Invictus)
* Reviews: [Befana](https://en.wikipedia.org/wiki/Befana)

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

###Got the game code from the Quixo code and just changed some stuff

In [26]:
from abc import ABC, abstractmethod
from copy import deepcopy
from enum import Enum
import numpy as np
import random as rand
from tqdm import tqdm

# Rules on PDF

class Player(ABC):
    def __init__(self) -> None:
        '''You can change this for your player if you need to handle state/have memory'''
        pass

    @abstractmethod
    def make_move(self, game: 'Game') -> tuple[tuple[int, int]]:
        '''
        game: the Tris game. You can use it to override the current game with yours, but everything is evaluated by the main game
        return values: this method shall return a tuple of X,Y positions
        '''
        pass


class Game(object):
    def __init__(self) -> None:
        self._board = np.ones((3, 3), dtype=np.uint8) * -1

    def get_board(self):
        '''
        Returns the board
        '''
        return deepcopy(self._board)

    def print(self):
        '''Prints the board. -1 are neutral pieces, 0 are pieces of player 0, 1 pieces of player 1'''
        print(self._board)

    def check_winner(self) -> int:
        '''Check the winner. Returns the player ID of the winner if any, otherwise returns -1'''
        # for each row
        for x in range(self._board.shape[0]):
            # if a player has completed an entire row
            if self._board[x, 0] != -1 and all(self._board[x, :] == self._board[x, 0]):
                # return the relative id
                return self._board[x, 0]
        # for each column
        for y in range(self._board.shape[1]):
            # if a player has completed an entire column
            if self._board[0, y] != -1 and all(self._board[:, y] == self._board[0, y]):
                # return the relative id
                return self._board[0, y]
        # if a player has completed the principal diagonal
        if self._board[0, 0] != -1 and all(
            [self._board[x, x]
                for x in range(self._board.shape[0])] == self._board[0, 0]
        ):
            # return the relative id
            return self._board[0, 0]
        # if a player has completed the secondary diagonal
        if self._board[0, -1] != -1 and all(
            [self._board[x, -(x + 1)]
             for x in range(self._board.shape[0])] == self._board[0, -1]
        ):
            # return the relative id
            return self._board[0, -1]
        return -1

    def play(self, player1: Player, player2: Player) -> int:
        '''Play the game. Returns the winning player'''
        players = [player1, player2]
        current_player_idx = 1
        winner = -1
        while winner < 0:
            if -1 not in self.get_board().ravel():
              winner = -2
              break
            current_player_idx += 1
            current_player_idx %= len(players)
            ok = False
            while not ok:
                from_pos = players[current_player_idx].make_move(self)
                ok = self._move(from_pos, current_player_idx)
            winner = self.check_winner()
        return winner

    def _move(self, from_pos: tuple[int, int], player_id: int) -> bool:
        '''Perform a move'''
        if player_id > 2:
            return False
        # Oh God, Numpy arrays
        prev_value = deepcopy(self._board[(from_pos[1], from_pos[0])])

        acceptable = False
        if self._board[from_pos] < 0:
          self._board[from_pos] = player_id
          acceptable = True
        return acceptable

###Added a subclass for training

In [27]:
class TrainingGame(Game):
  def __init__(self) -> None:
    super().__init__()

  def play(self, player1: Player, player2: Player) -> int:
        '''Play the game. Returns the winning player'''
        players = [player1, player2]
        current_player_idx = 1
        winner = -1

        while winner < 0:
            if -1 not in self.get_board().ravel():
              winner =-2
              break
            #get board old state
            old_state = self.get_board()

            current_player_idx += 1
            current_player_idx %= len(players)
            ok = False
            while not ok:
                from_pos = players[current_player_idx].make_move(self)
                ok = self._move(from_pos, current_player_idx)
                if not ok and isinstance(players[current_player_idx], MyPlayer):
                  #my model contains also invalid moves so I wanna learn not to do them
                  invalid_move_state = self.get_board()
                  reward = players[current_player_idx].compute_reward(is_winner=-1 , new_state=invalid_move_state)
                  #print("rew", reward)
                  players[current_player_idx].get_experience(old_state, from_pos, reward, invalid_move_state, False )

            winner = self.check_winner()

            if isinstance(players[current_player_idx], MyPlayer):
              #get board new state
              new_state = self.get_board()
              #1 here the player should get the reward
              winval = 1 if winner == current_player_idx else (0 if winner == (current_player_idx + 1) % len(players) else -1)
              reward = players[current_player_idx].compute_reward( winval, new_state)
              #print("rew", reward, "winner", winner)
              #2 here I would be capable of gettin my new step record, to put it into my replay memory
              players[current_player_idx].get_experience(old_state, from_pos, reward, new_state, winner!=-1 )
        return winner

In [28]:
class RandomPlayer(Player):
    def __init__(self) -> None:
        super().__init__()

    def make_move(self, game: 'Game') -> tuple[tuple[int, int]]:
        from_pos = (rand.randint(0, 2), rand.randint(0, 2))
        return from_pos

###This is my model: I'll put it inside my player

###It's a double Q Learning Model based with Replay Memory

###MyPlayer can be created as untrained, and then it goes completely random.
###If created as trained, it will start training against RandomPlayer.
###Replay Memory uncorrelates discovery from learning, making exploration less biased by the learning policy.
###The usage of two different tables, for extimation and as target, avoid the model following its own tail, getting learning faster, if tuned properly.
###I also use a Discount Factor and a Learning Rate, that I unforgivably keep constant.

In [56]:
class MyQTable():
  def __init__(self, num_states, num_actions) -> None:
    super().__init__()
    self.table = np.zeros((num_states, num_actions), dtype=float)

  def model_action(self, states):
    return np.array([deepcopy(self.table[index]) for index in states])

class MyPlayer(Player):
    def __init__(self, trained: bool = False) -> None:
        super().__init__()
        #for the sake of defining stuff
        self.last_status = None
        self.replay_mem = {}
        self.REPLAY_MEM_SIZE = 10_000
        self.exploration_rate = None
        self.model = None

        # if I want to use a trained model
        if trained:
            self.model = self.train()
        # if I don't want to use a trained model
        else:
            self.make_move = self.make_move_random

    def compute_reward(self, is_winner, new_state=None):
      #notto eleganto way to get the reward: winner should be decoupled but I had to check stuff twice...
      new_state = np.ravel(new_state)
      if is_winner==1:
        return 10
      #this if I make a not allowed move
      elif self.last_status is not None and self.last_status is not None and np.all(self.last_status == new_state):
        return -1
      elif is_winner == 0:
        return -10
      else:
        return 0

    def get_experience(self, old_state, action, reward, new_state, is_terminal):
      action = 3 * action[0] + action[1]
      old_state = self.get_qt_id(np.ravel(old_state))
      new_state = self.get_qt_id(np.ravel(new_state))
      key = (old_state, new_state)
      self.replay_mem[key] = tuple([old_state, action, reward, new_state, is_terminal])

      while len(self.replay_mem) >= self.REPLAY_MEM_SIZE:
          oldest_key = next(iter(self.replay_mem))
          del self.replay_mem[oldest_key]

    def get_qt_id(self, state):
      model_input = 0
      for tile in state:
        model_input *= 3
        model_input += 0 if tile == -1 else (1 if tile == 0 else 2)
      return model_input

    def make_move(self, game: 'Game') -> tuple[tuple[int, int]]:
        #I preprocess my input by flattening the state
        board_copy = game.get_board()
        board_copy = np.ravel(board_copy)

        #if I have already tried my deterministic choice
        # and the state is the same, it wasn't allowed so I'll go random or get stuck
        if np.all(self.last_status == board_copy):
            return self.make_move_random(game)
        self.last_status = board_copy

        if rand.random() > self.exploration_rate:
          from_pos = self.model.model_action(np.array([self.get_qt_id(board_copy)])).argmax()
        else:
          return self.make_move_random(game)

        #bring the format back to the one of the game
        if from_pos < 3:
            from_pos = tuple([0, from_pos])
        elif 2 < from_pos < 6:
            from_pos = tuple([1, from_pos - 3])
        else:
            from_pos = tuple([2, from_pos - 6])

        return from_pos

    def make_move_random(self, game: 'Game') -> tuple[tuple[int, int]]:
        from_pos = (rand.randint(0, 2), rand.randint(0, 2))
        return from_pos

    def train(self, num_episodes = 800_000, player2=RandomPlayer()) -> MyQTable:
        if self.model is None:
          new_model = MyQTable(3**9, 9)
          self.model = new_model
        if True:
            #setup the training
            DISCOUNT_RATE = 0.99
            TARGET_UP_RATE = 50
            learning_rate = 0.01
            MAX_EX_RATE = 1
            MIN_EX_RATE = 0.01
            EX_RATE_DECAY = 0.001
            self.exploration_rate = MAX_EX_RATE

            #accessory data structure for training
            self.replay_mem = {}
            target_model = deepcopy(self.model)

            for episode in tqdm(range(num_episodes)):
                mygame = TrainingGame()
                #print("episode ", episode)
                if episode % 2 == 0:
                    winner = mygame.play(player2, self)
                else:
                    winner = mygame.play(self, player2)
                if episode % 2 == winner:
                    pass
                #print("winner",winner + 1)

                #get a batch from replay memory and update the model
                # batch = rand.choices(self.replay_mem, k=32)
                # states, actions, rewards, next_states, dones = zip(*batch)
                # rewards = np.array(rewards)
                # dones = np.array(dones)

                random_keys = rand.choices(list(self.replay_mem.keys()), k=32)
                states, actions, rewards, next_states, dones = [], [], [], [], []
                for key in random_keys:
                  random_obs = self.replay_mem[key]
                  states.append(random_obs[0])
                  actions.append(random_obs[1])
                  rewards.append(random_obs[2])
                  next_states.append(random_obs[3])
                  dones.append(random_obs[4])
                rewards = np.array(rewards)
                dones = np.array(dones)

                #q values of the current state s_i
                q_values_a1 = self.model.model_action(states)

                #uses a1_i to get just the q values of interest
                q_values_selected_a1 = np.array([q_values_a1[i][action] for i, action in enumerate(actions)])

                #target q values using the Bellman equation
                target_q_values_next_states_a1 = target_model.model_action(next_states)

                #select the maximum q value for each action in the next state
                max_q_values_next_states_a1 = np.array([max(inner_list) for inner_list in target_q_values_next_states_a1])

                #get target q values using the Bellman equation
                target_q_values_a1 = rewards + (1 - dones) * DISCOUNT_RATE * max_q_values_next_states_a1

                #update the model
                for i, batch_element in enumerate(zip(states, actions)):
                  mystate, myaction = batch_element
                  self.model.table[mystate, myaction] = (1 - learning_rate) * q_values_selected_a1[i] + learning_rate *  target_q_values_a1[i]

                #once in a while I update the target network
                if episode % TARGET_UP_RATE == 0:
                    target_model = deepcopy(self.model)
                #update the exploration rate
                self.exploration_rate = MIN_EX_RATE + (MAX_EX_RATE - MIN_EX_RATE) * np.exp(-EX_RATE_DECAY*episode)
        self.exploration_rate = 0
        return self.model


###This wants to be a sort of Curriculum Learning:
###My model also consider invalid moves, so it has to learn to play at first.
###It's not really CL because I'm not changing the reward function but I am using a random player as opponent to maximize variability in the scenarios that my agent will come through.

In [57]:
g = Game()
player1 = MyPlayer(trained=True)
player2 = RandomPlayer()
player3 = deepcopy(player1)
print(player1.model.table[0])
print(player1.replay_mem)

100%|██████████| 800000/800000 [22:15<00:00, 598.87it/s]


[9.60596004 9.60595999 9.60595995 9.60596003 9.60596    9.60595999
 9.60596004 9.60595827 9.60596002]
{(248, 248): (248, 8, -1, 248, False), (248, 1706): (248, 2, 0, 1706, False), (8267, 8321): (8267, 5, 10, 8321, True), (4617, 4644): (4617, 5, 0, 4644, False), (4650, 5379): (4650, 2, 0, 5379, False), (18501, 18502): (18501, 8, 10, 18502, True), (516, 4890): (516, 1, 0, 4890, False), (5619, 5619): (5619, 1, -1, 5619, False), (5619, 5621): (5619, 8, 0, 5621, False), (5702, 5702): (5702, 8, -1, 5702, False), (5702, 18824): (5702, 0, 0, 18824, False), (7047, 7056): (7047, 6, 0, 7056, False), (8514, 8541): (8514, 5, 0, 8541, False), (8547, 8628): (8547, 4, 0, 8628, False), (2205, 8766): (2205, 0, 0, 8766, False), (9252, 9253): (9252, 8, 0, 9253, False), (9307, 9307): (9307, 6, -1, 9307, False), (9307, 9388): (9307, 4, 10, 9388, True), (13152, 13638): (13152, 3, 0, 13638, False), (14367, 14367): (14367, 3, -1, 14367, False), (14367, 14385): (14367, 6, 10, 14385, True), (13149, 13392): (1314

In [49]:
print(player1.model.table[10626])

[0. 0. 0. 0. 0. 0. 0. 0. 0.]


###I set Exploration Rate to .1 to my opponent because it will make it play 10% of the time random, being it a copy of my trained agent.
###I also tried to let it be fully exploitative but then my model starts going worse against the RandomPlayer.

In [58]:
g = Game()
player3.exploration_rate = .1
player1.train(player2=player3)
print(player1.model.table[0])
print(player1.replay_mem)

100%|██████████| 800000/800000 [20:55<00:00, 637.25it/s]

[9.6059601 9.6059601 9.6059601 9.6059601 9.6059601 9.6059601 9.6059601
 9.6059601 9.6059601]
{(9, 13131): (9, 0, 0, 13131, False), (13132, 14590): (13132, 2, 0, 14590, False), (0, 729): (0, 2, 0, 729, False), (783, 786): (783, 7, 0, 786, False), (1272, 1272): (1272, 3, -1, 1272, False), (1272, 3459): (1272, 1, 0, 3459, False), (9, 1467): (9, 2, 0, 1467, False), (1710, 1764): (1710, 5, 0, 1764, False), (0, 2187): (0, 1, 0, 2187, False), (2205, 2286): (2205, 4, 0, 2286, False), (2292, 2319): (2292, 5, 0, 2319, False), (3777, 3777): (3777, 6, -1, 3777, False), (3777, 4020): (3777, 3, 10, 4020, True), (9, 9): (9, 6, -1, 9, False), (13132, 17506): (13132, 1, 0, 17506, False), (0, 27): (0, 5, 0, 27, False), (13149, 13150): (13149, 8, 0, 13150, False), (13168, 13168): (13168, 5, -1, 13168, False), (13168, 13897): (13168, 2, 10, 13897, True), (9, 63): (9, 5, 0, 63, False), (306, 4680): (306, 1, 0, 4680, False), (0, 9): (0, 6, 0, 9, False), (4383, 4383): (4383, 1, -1, 4383, False), (4383, 4626)

###And this is how it goes against random:

In [59]:
win1 = 0
draw1 = 0
plays = 3_000
for i in range(plays):
  g = Game()
  winner = g.play(player1, player2)
  if winner == 0:
    win1 += 1
  elif winner == -2:
    draw1 += 1
print("Win perc as 1st:", win1 * 100 / plays)
print("Draw perc as 1st:", draw1 * 100 / plays)

Win perc as 1st: 94.26666666666667
Draw perc as 1st: 0.6666666666666666


In [61]:
win2 = 0
draw2 = 0
plays = 3_000
for i in range(plays):
  g = Game()
  winner = g.play(player2, player1)
  if winner == 1:
    win2 += 1
  elif winner == -2:
    draw2 += 1
print("Win perc as 1st:", win2 * 100 / plays)
print("Draw perc as 1st:", draw2 * 100 / plays)

Win perc as 1st: 62.733333333333334
Draw perc as 1st: 0.0


###And here against player3 that I make fully exploitative again...not really a great result, after all :\\

In [62]:
player3.exploration_rate = 0

win1 = 0
draw1 = 0
plays = 3_000
for i in range(plays):
  g = Game()
  winner = g.play(player1, player3)
  if winner == 0:
    win1 += 1
  elif winner == -2:
    draw1 += 1
print("Win perc as 1st:", win1 * 100 / plays)
print("Draw perc as 1st:", draw1 * 100 / plays)

Win perc as 1st: 100.0
Draw perc as 1st: 0.0


In [63]:
win2 = 0
draw2 = 0
plays = 3_000
for i in range(plays):
  g = Game()
  winner = g.play(player3, player1)
  if winner == 1:
    win2 += 1
  elif winner == -2:
    draw2 += 1
print("Win perc as 1st:", win2 * 100 / plays)
print("Draw perc as 1st:", draw2 * 100 / plays)

Win perc as 1st: 0.0
Draw perc as 1st: 0.0
